# Module 5 - Programming Assignment

In [1]:
from IPython.core.display import *
from StringIO import StringIO
import random
from random import gauss
import copy

## Local Search - Genetic Algorithm

This program uses the Genetic Algorithm to find the solution to a shifted Sphere Function in 10 dimensions, $x$, where the range of $x_i$ in each dimension is (-5.12 to 5.12). Here a "solution" means the vector $x$ that minimizes the function. The Sphere Function is:

$$f(x)=\sum x^2_i$$

We are going to shift it over 0.5 in every dimension:

$$f(x) = \sum (x_i - 0.5)^2$$

where $n = 10$.

As this *is* a minimization problem you'll the program uses the trick described in the lecture to turn the shifted Sphere Function into an appropriate fitness function (which is always looking for a *maximum* value).

## Binary GA

The problem is solved in two different ways. First, using the traditional (or "Canonical") Genetic Algorithm that encodes numeric values as binary strings (represented as lists of only 0 or 1).

There are many different ways to affect this encoding. This program uses a 10 bit binary encoding for each $x_i$. This gives each $x_i$ a potential value of 0 to 1024 which is be mapped to (-5.12, 5.12) by subtracting 512 and dividing by 100.

All the GA operators are as described in the lecture.

**Important**

There is a difference between the *genotype* and the *phenotype*. The GA operates on the *genotype* (the encoding) and does not respect the boundaries of the phenotype (the decoding). So, for example, do **not** use a List of Lists to represent an individual. It should be a *single* List of 10 x 10 or 100 bits. In general, crossover and mutation have no idea what those bits represent.

## Real Valued GA

For the real valued GA, each $x_i$ is represented as a float in the range (-5.12, 5.12) and the mutation operator applies gaussian noise. Python's random number generator for the normal distribution is called `gauss` and is found in the random module:

```
from random import gauss, random
```

You may need to experiment a bit with the standard deviation of the noise but the mean will be 0.0.

## GA

The Genetic Algorithm itself will have the same basic structure in each case: create a population, evaluate it, select parents, apply crossover and mutation, repeat until the number of desired generations have been generated. The easiest way to accomplish this in "Functional" Python would be to use Higher Order Functions.



Your code should print out the best individual of each generation including the generation number, genotype (the representation), phenotype (the actual value), the fitness (based on your fitness function transformation) and the function value (for the shifted sphere) if passed a DEBUG=True flag.

The GA has a lot of parameters: mutation rate, crossover rate, population size, dimensions (given for this problem), number of generations.  You can put all of those and your fitness function in a `Dict` in which case you need to implement:

```python
def binary_ga( parameters):
  pass
```

and

```python
def real_ga( parameters):
  pass
```

Remember that you need to transform the sphere function into a legit fitness function. Since you also need the sphere function, I would suggest that your parameters Dict includes something like:

```python
parameters = {
   "f": lambda xs: sphere( 0.5, xs),
   "minimization": True
   # put other parameters in here.
}
```

and then have your code check for "minimization" and create an entry for "fitness" that is appropriate.

In [2]:
def sphere( shift, xs):
    return sum( [(x - shift)**2 for x in xs])

In [3]:
sphere( 0.5, [1.0, 2.0, -3.4, 5.0, -1.2, 3.23, 2.87, -4.23, 3.82, -4.61])

113.42720000000001


-----

## Helper Functions

&nbsp;

**Minimization fitness**

minimization_fitness transforms the value returned from the sphere into an appropriate fitness function, since shifted Sphere Function is a minimization problem, and the fitness function is looking for a maximum.

In [4]:
def minimization_fitness(fitness_score):
    return 1 / (1 + fitness_score)

&nbsp;

**Initialize phenotype**

initialize_phenotype initializes a random phenotype for an individual.  It takes the given dimensions (10 for this problem) and initializes a list of 10 random numbers between -5.12 and 5.12.  This function is used to initialize the entire population.

In [5]:
def initialize_phenotype(dimensions):
    phenotype = []
    for i in range(dimensions):
        phenotype.append(round(random.uniform(-5.12, 5.12), 2))
    return phenotype

&nbsp;

**Binary GA phenotype to genotype**

It is necessary to be able to convert from phenotype to genotype for the binary GA.  This is used when the population is initialized.  Each genotype is initialized by converting the created phenotype to the corresponding genotype using the 10 bit binary encoding for each $x_i$. This gives each $x_i$ a potential value of 0 to 1024 which is be mapped to (-5.12, 5.12) by subtracting 512 and dividing by 100.  The genotype is represented as a list of 0s and 1s.

In [6]:
def binary_ga_phenotye_to_genotype(phenotype):
    genotype = []
    for decimal_representation in phenotype:
        decimal_representation = int((decimal_representation * 100) + 512)
        binary_representation = [int(x) for x in bin(decimal_representation)[2:]]
        for bit in binary_representation:
            genotype.append(bit)
    return genotype

&nbsp;

**Binary GA genotype to phenotype**

It is necessary to be able to convert from genotype to phenotype for the binary GA.  This is used in the binary GA crossover function, each time a new genotype is created using crossover to store the corresponding phenotype.  binary_ga_genotype_to_phenotype converts the binary representation of each dimension to the decimal representation, then subtracts 512 and divides by 100 to account for the binary encoding procedure described above.  

In [7]:
def binary_ga_genotype_to_phenotype(genotype):
    split_genotype = [genotype[i:i + 10] for i in xrange(0, len(genotype), 10)]
    phenotype = []
    for binary_representation in split_genotype:
        decimal_representation = int("".join(str(x) for x in binary_representation), 2)
        decimal_representation = (decimal_representation - 512.0) / 100
        phenotype.append(decimal_representation)
    return phenotype

&nbsp;

**Binary GA initialize population**

This function initializes the population for the binary GA with N random individuals of M dimensions, where N and M are defined as "population_size" and "dimensions" in the parameters dict passed to the genetic algorithm.  This function uses intialize_phenotype for each individual's phenotype, and binary_ga_phenotye_to_genotype to create its genotype.  The population is a list of dictionaries representing individuals.  Each dictionary contains key "phenotype" and "genotype", and this dictionary later includes key "fitness".

In [8]:
def binary_ga_initialize_population(population_size, dimensions):
    population = []
    for i in range(population_size):
        individual = {}
        phenotype = initialize_phenotype(dimensions)
        individual["phenotype"] = phenotype
        individual["genotype"] = binary_ga_phenotye_to_genotype(phenotype)
        population.append(individual)

    return population

&nbsp;

**Real GA initialize population**

This function initializes the population for the real GA with N random individuals of M dimensions, where N and M are defined as "population_size" and "dimensions" in the parameters dict passed to the genetic algorithm.  This function uses intialize_phenotype for each individual's phenotype, and and copies this value to create its genotype.  The population is a list of dictionaries representing individuals.  Each dictionary contains key "phenotype" and "genotype", and this dictionary later includes key "fitness".

In [9]:
def real_ga_initialize_population(population_size, dimensions):
    population = []
    for i in range(population_size):
        individual = {}
        phenotype = initialize_phenotype(dimensions)
        individual["phenotype"] = phenotype
        individual["genotype"] = copy.deepcopy(phenotype)
        population.append(individual)

    return population

&nbsp;

**Calculate fitness**

This function calculates the fitness of an individual, given the dict representing the individual as described above and a flag that is True if this is a minimization problem, false otherwise.  The calculation is done on the phenotype of the individual.  If this is not a minimization problem, the sphere function is used as the fitness function.  If it is a minimization function, the minimization_fitness function is run on the value produced by the sphere function.  calculate_fitness is used to evaluate the entire population. The dict representing the individual is updated with keys "fitness" and "shifted_sphere_value."

In [10]:
def calculate_fitness(individual, minimization):
    phenotype = individual["phenotype"]
    shifted_sphere_value = sphere(0.5, phenotype)
    individual["shifted_sphere_value"] = shifted_sphere_value
    
    if minimization:
        fitness = minimization_fitness(shifted_sphere_value)
    
    individual["fitness"] = fitness


&nbsp;

**Evaluate population**

This function applies the fitnss function to each individual in a given population, and returns the best individual in terms of fitness from that population.  It adds a new field "fitness" to the dictionary representing the individual.  It also takes a minimization flag to pass to the calculate_fitness function.  This is used in the main genetic algorithm to evaluate the population's fitness before parents are selected, and to allow the main algorithm to keep track of the best individual overall.

In [11]:
def evaluate_population(population, minimization):
    best_fitness = float("-inf")
    best_individual = None
    for individual in population:
        calculate_fitness(individual, minimization)
        fitness = individual["fitness"]
        if fitness > best_fitness:
            best_individual = individual
            best_fitness = fitness

    return best_individual

&nbsp;

**Select parent**

Given a random selection of individuals from a population, select_parent finds the individual in that list with the best fitness and returns that individual.  This is a helper function for the tournament selection algorithm for selecting two parents from a population.

In [12]:
def select_parent(random_selection):
    max_fitness = float("-inf")
    for individual in random_selection:
        fitness = individual["fitness"]
        if fitness > max_fitness:
            max_fitness = fitness
            max_parent = individual

    return max_parent

&nbsp;

**Get random indices**

Given a population and a parameter tournament_selection_number representing the number of individuals to select for the tournament selection algorithm, get_random_indices selects tournament_selection_number of random indices in the population that will be used by the tournament selection algorithm to select random individuals from the population.

In [13]:
def get_random_indices(population, tournament_selection_number):
    random_indices = []
    while len(random_indices) < tournament_selection_number:
        random_index = random.randint(0, (len(population) - 1))
        if random_index not in random_indices:
            random_indices.append(random_index)
            
    return random_indices

&nbsp;

**Select parents - tournament selection**

Given a population, select_parents_tournament_selection selects some given number of individuals at random, then selects the one with the highest fitness.  It does so using the get_random_indices function, then selects those individuals from the population and picks the one with the best fitness.  It ensures that parent1 and parent2 are not the same by, if parent1 is in the random individuals list for parent2 selection, removing it.

In [14]:
def select_parents_tournament_selection(population, tournament_selection_number):
    random_indices = get_random_indices(population, tournament_selection_number)
     
    random_individuals = []       
    for index in random_indices:
        random_individuals.append(population[index])
    
    parent1 = select_parent(random_individuals)
    
    random_indices = get_random_indices(population, tournament_selection_number)
     
    random_individuals = []       
    for index in random_indices:
        random_individuals.append(population[index])

    if parent1 in random_individuals:
        random_individuals.remove(parent1)
    parent2 = select_parent(random_individuals)

    return parent1, parent2

&nbsp;

**Binary GA crossover**

Given two individuals representing parents, binary_ga_crossover implements crossover for the binary GA.  It gets a random crossover index by selecting a random integer between 0 and the length - 1 of a parent genotype. Child 1's genotype is built using 0:crossover_index + crossover_index:end of parent 1's genotype.  Child 2's genotype is built using 0:crossover_index + crossover_index:end of parent 2's genotype.  Both childrens' phenotypes are set using the binary_ga_genotype_to_phenotype function. The function returns child1, child2, and is used in the reproduce function in the binary GA.

In [15]:
def binary_ga_crossover(parent1, parent2):
    child1 = {}
    child2 = {}

    parent1_genotype = parent1["genotype"]
    parent2_genotype = parent2["genotype"]

    crossover_index = random.randint(0, len(parent1_genotype) - 1)

    child1_genotype = parent1_genotype[:crossover_index] + parent2_genotype[crossover_index:]
    child2_genotype = parent2_genotype[:crossover_index] + parent1_genotype[crossover_index:]

    child1["genotype"] = child1_genotype
    child2["genotype"] = child2_genotype

    child1["phenotype"] = binary_ga_genotype_to_phenotype(child1_genotype)
    child2["phenotype"] = binary_ga_genotype_to_phenotype(child2_genotype)

    return child1, child2

&nbsp;

**Real GA crossover**

Given two individuals representing parents, real_ga_crossover implements crossover for the real GA.  It gets a random crossover index by selecting a random integer between 0 and the length - 1 of a parent genotype. Child 1's genotype is built using 0:crossover_index + crossover_index:end of parent 1's genotype.  Child 2's genotype is built using 0:crossover_index + crossover_index:end of parent 2's genotype.  In this case, the phenotype is the same as the genotype, a list of floats between -5.12 and 5.12.  So, the phenotype of each child is set as the genotype just created. The function returns child1, child2, and is used in the reproduce function in the real GA.

In [16]:
def real_ga_crossover(parent1, parent2):
    child1 = {}
    child2 = {}

    parent1_genotype = parent1["genotype"]
    parent2_genotype = parent2["genotype"]

    crossover_index = random.randint(0, len(parent1_genotype) - 1)

    child1_genotype = parent1_genotype[:crossover_index] + parent2_genotype[crossover_index:]
    child2_genotype = parent2_genotype[:crossover_index] + parent1_genotype[crossover_index:]

    child1["genotype"] = child1_genotype
    child2["genotype"] = child2_genotype

    child1["phenotype"] = child1_genotype
    child2["phenotype"] = child2_genotype

    return child1, child2

&nbsp;

**Binary GA mutate**

Given an individual representing a child, binary_ga_mutate performs a mutation on that child.  It does so by finding a random location in the child's genotype, selecting a random value between 0 and 1, and inserting that value into the random location. This is used in the reproduce function of the binary GA.

In [17]:
def binary_ga_mutate(child, parameters):
    genotype = child["genotype"]
    mutation_site = random.randint(0, len(genotype) - 1)
    mutation = random.randint(0, 1)
    genotype[mutation_site] = mutation

&nbsp;

**Real GA mutate**

Given an individual representing a child, real_ga_mutate performs a mutation on that child.  It does so by first finding a random location in the child's genotype.  Then, it uses gaussian noise to produce a mutation with a mean of zero and standard deviation selected from the parameters dict.  I have set 1.7 as the standard deviation, since this is 5.12 / 3, so the majority of the values (99.7%) are within the desired range of -5.12 to 5.12.  If ever a mutation is selected outside this range, real_ga_mutate tries again until a valid value is found.  Once a mutation value is selected, it is inserted into the randomly generated location in the individual's genotype.  This is used in the reproduce function of the Real GA.

In [18]:
def real_ga_mutate(child, parameters):
    genotype = child["genotype"]
    mutation_site = random.randint(0, len(genotype) - 1)
    current_value = genotype[mutation_site]
    mutation value = current_value + gauss(0.0, parameters["sigma"])
    while mutation > 5.12 or mutation < -5.12:
        mutation = gauss(0, parameters["sigma"])
    genotype[mutation_site] = mutation

&nbsp;

**Reproduce**

Given two parents, the parameters dict, and crossover and mutate functions, reproduce performs reproduction as it is defined for the genetic algorithm.  The probability of crossover and mutation are part of the parameters dict.

1. Pick a random number, crossover_chance
2. If crossover_chance is less than the probability of crossover, return parent1, parent2
3. Otherwise, perform crossover
4. Pick a random number mutation_chance
5. If mutation chance is less than the probability of mutation, do not mutate, otherwise mutate
6. Repeat 4. and 5. for child two
7. Return child1, child2

In [19]:
def reproduce(parent1, parent2, parameters, crossover, mutate):
    crossover_chance = random.uniform(0, 1)
    crossover_rate = parameters["crossover_rate"]
    mutation_rate = parameters["mutation_rate"]
    if crossover_chance > crossover_rate:
        return parent1, parent2

    child1, child2 = crossover(parent1, parent2)

    child1_mutation_chance = random.uniform(0, 1)
    if child1_mutation_chance < mutation_rate:
        mutate(child1, parameters)

    child2_mutation_chance = random.uniform(0, 1)
    if child2_mutation_chance < mutation_rate:
        mutate(child2, parameters)

    return child1, child2

&nbsp;

**Print individual**

Given an individual, print the genotype (the representation), phenotype (the actual value), the fitness (based on the fitness function transformation) and the function value (for the shifted sphere).  This is used in the genetic algorithm if the DEBUG flag is set to True.

In [20]:
def print_individual(individual):
    print 'Individual:'
    print '\tgenotype:', individual["genotype"]
    print '\tphenotype:', individual["phenotype"]
    print '\tfitness:', individual["fitness"]
    print '\tshifted sphere function value:', individual["shifted_sphere_value"]
    print

---
## Genetic Algorithm

The Genetic Algorithm has the structure: create a population, evaluate it, select parents, apply crossover and mutation, repeat until the number of desired generations have been generated. This is accomplished using Higher Order Functions initialize_population, crossover, mutate.

The best individual of each generation is printed including the generation number, genotype (the representation), phenotype (the actual value), the fitness (based on the fitness function transformation) and the function value (for the shifted sphere) if passed a DEBUG=True flag.  It prints approximately 50 individuals by calculating print_frequency = the number of desired generations / 50 and only printing an individual if the current number of generations % print_frequency is equal to 0.

As the GA has a lot of parameters: mutation rate, crossover rate, population size, dimensions (given for this problem), number of generations, so all are stored in a `Dict` parameters, which is passed to the algorithm in addition to the functions listed above.

The GA returns the best individual, in terms of fitness, ever encountered.

Real GA and Binary GA simply call the genetic algorithm with the appropriate parameters and functions.

In [21]:
def genetic_algorithm(parameters, initialize_population, crossover, mutate, DEBUG=False):
    population_size = parameters["population_size"]
    population = initialize_population(population_size, parameters["dimensions"])
    best_individual = { "fitness": float("-inf") }

    generations = 0
    NUMPRINT = 0
    print_frequency = parameters["number_of_generations"] / 50
    while generations < parameters["number_of_generations"]:
        candidate_best_individual = evaluate_population(population, parameters["minimization"])  # each individual gets a fitness score before we go to pick parents
        if DEBUG and (generations % print_frequency == 0):
            print 'Generation number:', generations
            print_individual(candidate_best_individual)
            NUMPRINT +=1
            
        if candidate_best_individual["fitness"] > best_individual["fitness"]:
            best_individual = candidate_best_individual

        next_population = []
        for i in range(population_size / 2):
            parent1, parent2 = select_parents_tournament_selection(population, parameters["tournament_selection_number"])
            child1, child2 = reproduce(parent1, parent2, parameters, crossover, mutate)

            next_population.append(child1)
            next_population.append(child2)

        population = next_population
        generations += 1

    return best_individual

In [22]:
def binary_ga(parameters):
    return genetic_algorithm(parameters, binary_ga_initialize_population, binary_ga_crossover, binary_ga_mutate, DEBUG=True)

In [23]:
def real_ga(parameters):
    return genetic_algorithm(parameters, real_ga_initialize_population, real_ga_crossover, real_ga_mutate, DEBUG=True)

---
## Test Binary GA

In [24]:
binary_ga_parameters = {
    "f": lambda xs: sphere(0.5, xs),
    "minimization": True,  
    "mutation_rate": .05,
    "crossover_rate": .9,
    "population_size": 10000, 
    "dimensions": 10, 
    "number_of_generations": 75,  
    "tournament_selection_number": 7
}
best_individual = binary_ga(binary_ga_parameters)
print '\n\nBINARY GENETIC ALGORITHM BEST INDIVIDUAL'
print_individual(best_individual)

Generation number: 0
Individual:
	genotype: [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0]
	phenotype: [-0.12, 1.58, -0.55, 1.11, 1.61, 0.73, -1.16, 0.81, -0.67, 1.32]
	fitness: 0.0980065468373
	shifted sphere function value: 9.2034

Generation number: 1
Individual:
	genotype: [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1]
	phenotype: [0.14, 2.34, 0.69, 0.98, 0.53, 1.33, 0.74, 1.99, -1.21]
	fitness: 0.093691735452
	shifted sphere function value: 9.6733

Generation number: 2
Individual:
	genotype: [1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0

## Test Real GA

In [25]:
real_ga_parameters = {
    "f": lambda xs: sphere(0.5, xs),
    "minimization": True,  
    "mutation_rate": .05,
    "crossover_rate": .9,
    "population_size": 5000,
    "dimensions": 10, 
    "number_of_generations": 200,  
    "tournament_selection_number": 7,
    "sigma": 1.7
}
best_individual = real_ga(real_ga_parameters)
print '\n\nREAL GENETIC ALGORITHM BEST INDIVIDUAL'
print_individual(best_individual)

Generation number: 0
Individual:
	genotype: [1.31, 0.09, 2.25, -2.23, 1.58, 0.96, -0.2, 1.04, 2.58, -0.63]
	phenotype: [1.31, 0.09, 2.25, -2.23, 1.58, 0.96, -0.2, 1.04, 2.58, -0.63]
	fitness: 0.049745056585
	shifted sphere function value: 19.1025

Generation number: 4
Individual:
	genotype: [1.05, 0.55, -0.51, 0.48, 0.05, 0.95, 0.28, 0.28, 0.12, 0.54]
	phenotype: [1.05, 0.55, -0.51, 0.48, 0.05, 0.95, 0.28, 0.28, 0.12, 0.54]
	fitness: 0.336326640433
	shifted sphere function value: 1.9733

Generation number: 8
Individual:
	genotype: [0.19, 0.56, 0.34, 0.6, 0.41, 0.4, 0.47, 0.42, 0.23, 0.39]
	phenotype: [0.19, 0.56, 0.34, 0.6, 0.41, 0.4, 0.47, 0.42, 0.23, 0.39]
	fitness: 0.802761499558
	shifted sphere function value: 0.2457

Generation number: 12
Individual:
	genotype: [0.42, 0.5, 0.48, 0.51, 0.51, 0.4, 0.47, 0.42, 0.49412070634102795, 0.56]
	phenotype: [0.42, 0.5, 0.48, 0.51, 0.51, 0.4, 0.47, 0.42, 0.49412070634102795, 0.56]
	fitness: 0.972824567813
	shifted sphere function value: 0.0279